In [10]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [11]:
product_data = pd.read_csv('../data/market_integrates_100.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2     prices  \
0             1         1         5.433865         2.042325  21.721590   
1             1         2         5.907737         2.412709  18.586650   
2             1         3         5.943911         2.174390  20.086869   
3             1         4         5.329441         2.844997  20.126112   
4             1         5         5.083033         2.137079  22.903818   
..          ...       ...              ...              ...        ...   
995         100         6         5.360283         2.580602  24.693921   
996         100         7         5.958183         2.650134  22.406677   
997         100         8         5.184745         2.346844  19.703203   
998         100         9         5.223118         2.464959  20.661943   
999         100        10         5.019838         2.675947  23.255443   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0        14.701881  0.061575  0.4322

In [12]:
# Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
product_data_complete = product_data.rename(columns={'labor':'demand_instrument0', 'capital':'demand_instruments1', 'marginal_cost': 'demand_instrument2'})
print(product_data_complete)



     market_ids  firm_ids  characteristic1  characteristic2     prices  \
0             1         1         5.433865         2.042325  21.721590   
1             1         2         5.907737         2.412709  18.586650   
2             1         3         5.943911         2.174390  20.086869   
3             1         4         5.329441         2.844997  20.126112   
4             1         5         5.083033         2.137079  22.903818   
..          ...       ...              ...              ...        ...   
995         100         6         5.360283         2.580602  24.693921   
996         100         7         5.958183         2.650134  22.406677   
997         100         8         5.184745         2.346844  19.703203   
998         100         9         5.223118         2.464959  20.661943   
999         100        10         5.019838         2.675947  23.255443   

     demand_instrument2    shares   profits   markups  e_quantity  \
0             14.701881  0.061575  0.43223

In [13]:
# the endog regressor
X1_formulation = pyblp.Formulation('1 + prices + characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

product_formulations = (X1_formulation, X2_formulation)
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [14]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [15]:
pr_integration = pyblp.Integration('product', size=5)


In [16]:
mc_problem = pyblp.Problem(product_formulations, product_data_complete, integration=mc_integration)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


'demand_instruments1' was specified but not 'demand_instruments0'.


In [17]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [18]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1          2172         6603         0     +9.964034E-22                 +2.826727E-09  +1.000000E+00

Optimization completed after 00:00:01.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:02.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------